## Dicionário de Dados

| Variável | Definição                                      | Chave                                          |
|----------|------------------------------------------------|------------------------------------------------|
| survival | Sobrevivência                                  | 0 = Não, 1 = Sim                               |
| pclass   | Classe do Bilhete                              | 1 = 1ª, 2 = 2ª, 3 = 3ª                         |
| sex      | Sexo                                           |                                                |
| Age      | Idade em anos                                  |                                                |
| sibsp    | Número de irmãos / cônjuges a bordo do Titanic |                                                |
| parch    | Número de pais / filhos a bordo do Titanic     |                                                |
| ticket   | Número do Bilhete                              |                                                |
| fare     | Tarifa do Passageiro                           |                                                |
| cabin    | Número da Cabine                               |                                                |
| embarked | Porto de Embarque                              | C = Cherbourg, Q = Queenstown, S = Southampton |

pclass: Classes economicas: 
    * 1st = Upper,
    * 2nd = Middle,
    * 3rd = Lower
age: Idade estimada
sibsp: Irmãos e cônjuges a bordo.
    * Sibling = Irmão, irmã, meio-irmão, meia-irmã;;;
    * Spouse = Marido, esposa (amantes e noivos foram ignorados)
parch: Pais e filhos a bordo.
    * Parent = Mãe, pai
    * Child = Filha, filho, enteada, enteado
    * Algumas crianças viajaram apenas com uma babá, portanto parch = 0 para elas.

In [ ]:
# Parte do código teve IA como copiloto, principalmente na questão de autocompletar códigos e correção de erros.

In [ ]:
# Resultados das analises. 
# Os resultados abaixo mostram como sempre devemos nos basear em outras métricas alem da acuracia.

from IPython.display import Image

Image(filename='./output/resultado.png')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style('whitegrid')

# Machine learning
from sklearn import model_selection, metrics
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from catboost import cv

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier

In [ ]:
# Verificando os arquivos no diretório
import os

for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Testando a leitura do arquivo
train_data = pd.read_csv('input/train.csv')
train_data.describe()

In [ ]:
# Testando a leitura do arquivo
test_data = pd.read_csv('input/test.csv')
test_data.describe()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
# Salvar PassengerId para auxiliar na criação do arquivo que será enviado ao Kaggle
passengerId = test_data['PassengerId']
passengerId

In [ ]:
# Criando um novo dataset com os dados de treino e teste
titanic_df = pd.concat([train_data, test_data], sort=False)

In [ ]:
# Para auxiliar no momento separar o DF titanic_df em train e test
train_index = len(train_data)
test_index = len(titanic_df) - len(test_data)

In [ ]:
# Verificando o novo dataset
titanic_df.info()

In [ ]:
# Verificando os dados faltantes
titanic_df.isnull().sum()

In [ ]:
titanic_df.info()

In [ ]:
# Verificando a distribuição dos dados
titanic_df.describe()

In [ ]:
# Verificando a distribuição dos dados
train_data.describe()

In [ ]:
# Criando o df onde iremos processar os modelos de ML

df = pd.DataFrame()

In [ ]:
# Sobreviventes
titanic_df['Survived'].nunique()

In [ ]:
titanic_df['Survived'].unique()

In [ ]:
titanic_df['Survived'].isnull().sum()

In [ ]:
titanic_df['Survived'].value_counts()

In [ ]:
sns.countplot(data=titanic_df, x='Survived')

In [ ]:
# Criando uma função para ajudar no momento de visualizar as informações de cada coluna

def titanic_func(data, column, count=True):
    print(f'Quantidade de valores únicos: {data[column].nunique()}')
    print(f'\nQuais são os valores únicos: {data[column].unique()}')
    print(f'\nQuantidade de valores nulos: {data[column].isnull().sum()}')
    print(f'\nQuantidade por opção: \n{data[column].value_counts()}')

    if count:
        sns.countplot(data=data, x=column, hue='Survived')
    else:
        sns.displot(data[column], kde=True)


titanic_func(titanic_df, 'Survived')

In [ ]:
df['Survived'] = titanic_df['Survived']

In [ ]:
df

In [ ]:
### Pclass

In [ ]:
titanic_func(titanic_df, 'Pclass')

In [ ]:
df['Pclass'] = titanic_df['Pclass']
df.head()

In [ ]:
### Sex

In [ ]:
titanic_df['Sex'].unique()

In [ ]:
titanic_df['Sex'] = titanic_df['Sex'].replace(['female', 'male'], [1, 0])

In [ ]:
titanic_func(titanic_df, 'Sex')

In [ ]:
df['Sex'] = titanic_df['Sex']
df.head()

In [ ]:
### Age

In [ ]:
titanic_func(titanic_df, 'Age', count=False)

In [ ]:
# Cria uma cópia do dataframe original, mantendo apenas as colunas numéricas
numeric_df = titanic_df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()

In [ ]:
titanic_df[titanic_df['Pclass'] == 1]['Age'].mean()

In [ ]:
titanic_df[titanic_df['Pclass'] == 2]['Age'].mean()

In [ ]:
titanic_df[titanic_df['Pclass'] == 3]['Age'].mean()

In [ ]:
for i in sorted(numeric_df['Pclass'].unique()):
    print(f'Pessoas da {i}º classe tem em média {round(numeric_df[numeric_df["Pclass"] == i]["Age"].mean(), 2)} anos')

In [ ]:
titanic_df[titanic_df['Pclass'] == 1]['Age'].isnull().sum()

In [ ]:
titanic_df['Age'].isnull().sum()

In [ ]:
# Preenchendo os valores nulos com a média de idade de cada classe
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df.groupby('Pclass')['Age'].transform('mean'))


In [ ]:
titanic_df['Age'].isnull().sum()

In [ ]:
titanic_func(titanic_df, 'Age', count=False)

In [ ]:
df['Age'] = titanic_df['Age']

In [ ]:
df.head()

In [ ]:
### SibSp

In [ ]:
titanic_func(titanic_df, 'SibSp')

In [ ]:
df['SibSp'] = titanic_df['SibSp']
df.head()

In [ ]:
### Parch

In [ ]:
titanic_func(titanic_df, 'Parch')

In [ ]:
df['Parch'] = titanic_df['Parch']

In [ ]:
df.head()

In [ ]:
### FamilySize 

In [ ]:
titanic_df['FamilySize'] = titanic_df['SibSp'] + titanic_df['Parch'] + 1

In [ ]:
df['FamilySize'] = titanic_df['FamilySize']
df.head()

In [ ]:
titanic_func(titanic_df, 'FamilySize')

In [ ]:
### Fare

In [ ]:
titanic_func(titanic_df, 'Fare', count=False)

In [ ]:
titanic_df['Fare'].isnull().sum()

In [ ]:
# Preenchendo os valores nulos com a média de tarifa de cada classe
titanic_df['Fare'] = titanic_df['Fare'].fillna(titanic_df.groupby('Pclass')['Fare'].transform('mean'))

In [ ]:
titanic_df['Fare'].isnull().sum()

In [ ]:
titanic_func(titanic_df, 'Fare', count=False)

In [ ]:
# media da tarifa por classe
titanic_df.groupby('Pclass')['Fare'].mean()

In [ ]:
titanic_df.isnull().sum()

In [ ]:
df['Fare'] = titanic_df['Fare']
df.head()

In [ ]:
### Cabin

In [ ]:
titanic_df['Cabin'].isnull().sum()

In [ ]:
titanic_df['Cabin'].nunique()

In [ ]:
titanic_df['Cabin'].unique()

In [ ]:
titanic_df['Cabin'].value_counts()

In [ ]:
titanic_df['Cabin'].fillna('N', inplace=True)

In [ ]:
titanic_df['Cabin'].isnull().sum()

In [ ]:
titanic_df['Cabin'].nunique()

In [ ]:
titanic_df['Cabin'].unique()

In [ ]:
titanic_df['Cabin'].value_counts()

In [ ]:
titanic_df['Cabin'] = titanic_df['Cabin'].apply(lambda x: x[0])

In [ ]:
titanic_df['Cabin'].unique()

In [ ]:
titanic_df['Cabin'].value_counts()

In [ ]:
titanic_func(titanic_df, 'Cabin')

In [ ]:
df['Cabin'] = titanic_df['Cabin']

In [ ]:
df.head()

In [ ]:
### Embarked

In [ ]:
print(titanic_df[titanic_df.index.duplicated()])

In [ ]:
titanic_df = titanic_df.reset_index(drop=True)

In [ ]:
titanic_func(titanic_df, 'Embarked')

In [ ]:
titanic_df[titanic_df['Embarked'] == 'S']['Survived'].mean()

In [ ]:
titanic_df[titanic_df['Embarked'] == "S"]['Pclass'].mean()

In [ ]:
titanic_df[titanic_df['Embarked'] == 'C']['Survived'].mean()

In [ ]:
titanic_df[titanic_df['Embarked'] == "C"]['Pclass'].mean()

In [ ]:
titanic_df[titanic_df['Embarked'] == 'Q']['Survived'].mean()

In [ ]:
titanic_df[titanic_df['Embarked'] == "Q"]['Pclass'].mean()

In [ ]:
titanic_df['Embarked'].fillna('S', inplace=True)

In [ ]:
titanic_df['Embarked'].isnull().sum()

In [ ]:
titanic_func(titanic_df, 'Embarked')

In [ ]:
df['Embarked'] = titanic_df['Embarked']

In [ ]:
### Nome

In [ ]:
titanic_df.head()

In [ ]:
titanic_df['Name']

In [ ]:
titanic_df['Title'] = titanic_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
titanic_df['Title'].unique()

In [ ]:
titanic_df['Title'].value_counts()

In [ ]:
titanic_df['Title'].isnull().sum()

In [ ]:
titanic_df['Title'] = titanic_df['Title'].replace(
    ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [ ]:
titanic_df['Title'] = titanic_df['Title'].replace('Mlle', 'Miss')

In [ ]:
titanic_df['Title'] = titanic_df['Title'].replace('Ms', 'Miss')

In [ ]:
titanic_df['Title'] = titanic_df['Title'].replace('Mme', 'Mrs')

In [ ]:
titanic_df['Title'].unique()

In [ ]:
titanic_df['Title'].value_counts()

In [ ]:
titanic_func(titanic_df, 'Title')

In [ ]:
df['Title'] = titanic_df['Title']

In [ ]:
df.head()

In [ ]:
titanic_df.isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
# Verificando a correlação entre as variáveis numeŕicas

In [ ]:
numeric_df = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_df.corr()

In [ ]:
numeric_df.corr()

In [ ]:
# Verificando a correlação entre as variáveis categóricas

In [ ]:
categoric_df = df.select_dtypes(exclude=[np.number])

In [ ]:
categoric_df.head()

In [ ]:
categoric_df = pd.get_dummies(categoric_df)

In [ ]:
categoric_df.head()

In [ ]:
categoric_df.corr()

In [ ]:
# Verificando a correlação entre as variáveis numeŕicas e categóricas

In [ ]:
df = pd.concat([numeric_df, categoric_df], axis=1)

In [ ]:
df.head()

In [ ]:
df.corr()

In [ ]:
train = df[:train_index].copy()
test = df[test_index:].copy()

train['Survived'] = train['Survived'].astype(int)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
X = train.drop('Survived', axis=1)
y = train['Survived']

In [ ]:
X_test = test.drop('Survived', axis=1)

In [ ]:
### Testando varios modelos

In [ ]:
def func_acuracia(algoritmo, X_train, y_train, vc):
    modelo = algoritmo.fit(X_train, y_train)
    acuracia = round(modelo.score(X_train, y_train) * 100, 2)

    train_pred = model_selection.cross_val_predict(algoritmo, X_train, y_train, cv=vc, n_jobs=-1)
    acuracia_vc = round(metrics.accuracy_score(y_train, train_pred) * 100, 2)

    return acuracia, acuracia_vc

In [ ]:
from sklearn.metrics import recall_score

from sklearn.metrics import f1_score


def func_acuracia_GS(algoritmo, params, X_train, y_train, cv):
    # Instanciando a GridSearchCV
    grid = GridSearchCV(algoritmo, params, cv=cv, n_jobs=-1, verbose=1)

    # Ajustando a GridSearchCV nos dados de treinamento
    grid.fit(X_train, y_train)

    # Selecionando o melhor estimador
    modelo = grid.best_estimator_

    # Verificando a acurácia nos dados de treinamento
    acuracia = round(modelo.score(X_train, y_train) * 100, 2)

    # Obtendo as previsões via cross-validation para verificar a acuracia
    train_pred = cross_val_predict(modelo, X_train, y_train, cv=cv, n_jobs=-1)

    acuracia_cv = round(accuracy_score(y_train, train_pred) * 100, 2)

    # Calcular Recall
    recall = round(recall_score(y_train, train_pred, average='macro') * 100,
                   2)  # Para labels binários pode remover "average"

    # Calcular F1-score
    f1 = round(f1_score(y_train, train_pred, average='macro') * 100, 2)  # Para labels binários pode remover "average"

    print(f"Melhores parâmetros: {grid.best_params_}")
    print(f"Acurácia após GridSearchCV: {acuracia}")
    print(f"Acurácia de validação cruzada: {acuracia_cv}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")

    return modelo, acuracia, acuracia_cv, recall, f1

In [ ]:
### Regressão Logística

In [ ]:
acc_log, acc_log_vc = func_acuracia(LogisticRegression(max_iter=1000), X, y, 10)

print(f'Acurácia Regressão Logística: {acc_log}')
print(f'Acurácia Regressão Logística com Validação Cruzada: {acc_log_vc}')

In [ ]:
### Regressao Logistica com GridSearchCV

In [ ]:
log_params = {'penalty': ['l2', 'elasticnet', 'none'],
              'C': [0.1, 0.5, 1, 5, 10],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'max_iter': [1000, 10000, 25000, 50000]}

modelo, acc_log, acc_log_cv, recall_log, f1_log = func_acuracia_GS(LogisticRegression(), log_params, X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia Regressão Logística: {acc_log}')
print(f'Acurácia Regressão Logística com Validação Cruzada: {acc_log_cv}')
print(f'Recall Regressão Logística: {recall_log}')
print(f'F1-Score Regressão Logística: {f1_log}')

In [ ]:
### KNN

In [ ]:
acc_knn, acc_knn_vc = func_acuracia(KNeighborsClassifier(n_neighbors=3), X, y, 10)

print(f'Acurácia KNN: {acc_knn}')
print(f'Acurácia KNN com Validação Cruzada: {acc_knn_vc}')

In [ ]:
### KNN com GridSearchCV

In [ ]:
knn_params = {'n_neighbors': [3, 5, 10, 20],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan', 'minkowski']}

modelo, acc_knn, acc_knn_cv, recall_knn, f1_knn = func_acuracia_GS(KNeighborsClassifier(), knn_params, X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia KNN: {acc_knn}')
print(f'Acurácia KNN com Validação Cruzada: {acc_knn_cv}')
print(f'Recall KNN: {recall_knn}')
print(f'F1-Score KNN: {f1_knn}')

In [ ]:
### Gaussian Naive Bayes

In [ ]:
acc_gaussian, acc_gaussian_vc = func_acuracia(GaussianNB(), X, y, 10)

print(f'Acurácia Gaussian Naive Bayes: {acc_gaussian}')
print(f'Acurácia Gaussian Naive Bayes com Validação Cruzada: {acc_gaussian_vc}')

In [ ]:
### Gaussian Naive Bayes com GridSearchCV

In [ ]:
gaussian_params = {'var_smoothing': np.logspace(0, -9, num=100)}

modelo, acc_gaussian, acc_gaussian_cv, recall_gaussian, f1_gaussian = func_acuracia_GS(GaussianNB(), gaussian_params,
                                                                                       X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia Gaussian Naive Bayes: {acc_gaussian}')
print(f'Acurácia Gaussian Naive Bayes com Validação Cruzada: {acc_gaussian_cv}')
print(f'Recall Gaussian Naive Bayes: {recall_gaussian}')
print(f'F1-Score Gaussian Naive Bayes: {f1_gaussian}')

In [ ]:
### Linear SVC

In [ ]:
acc_linear_svc, acc_linear_svc_vc = func_acuracia(LinearSVC(max_iter=10000), X, y, 10)

print(f'Acurácia Linear SVC: {acc_linear_svc}')
print(f'Acurácia Linear SVC com Validação Cruzada: {acc_linear_svc_vc}')

In [ ]:
### Linear SVC com GridSearchCV

In [ ]:
svc_params = {'C': [0.1, 0.5, 1, 5, 10],
              'max_iter': [1000, 10000, 25000, 50000]}

modelo, acc_svc, acc_svc_cv, recall_svc, f1_svc = func_acuracia_GS(LinearSVC(), svc_params, X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia Linear SVC: {acc_svc}')
print(f'Acurácia Linear SVC com Validação Cruzada: {acc_svc_cv}')
print(f'Recall Linear SVC: {recall_svc}')
print(f'F1-Score Linear SVC: {f1_svc}')

In [ ]:
### Stochastic Gradient Descent

In [ ]:
acc_sgd, acc_sgd_vc = func_acuracia(SGDClassifier(max_iter=1000), X, y, 10)

print(f'Acurácia Stochastic Gradient Descent: {acc_sgd}')
print(f'Acurácia Stochastic Gradient Descent com Validação Cruzada: {acc_sgd_vc}')

In [ ]:
### Stochastic Gradient Descent com GridSearchCV

In [ ]:
sgd_params = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['none', 'l1', 'l2', 'elasticnet']
}

modelo, acc_sgd, acc_sgd_cv, recall_sgd, f1_sgd = func_acuracia_GS(SGDClassifier(), sgd_params, X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia Stochastic Gradient Descent: {acc_sgd}')
print(f'Acurácia Stochastic Gradient Descent com Validação Cruzada: {acc_sgd_cv}')
print(f'Recall Stochastic Gradient Descent: {recall_sgd}')
print(f'F1-Score Stochastic Gradient Descent: {f1_sgd}')

In [ ]:
### Decision Tree

In [ ]:
acc_decision_tree, acc_decision_tree_vc = func_acuracia(DecisionTreeClassifier(), X, y, 10)

print(f'Acurácia Decision Tree: {acc_decision_tree}')
print(f'Acurácia Decision Tree com Validação Cruzada: {acc_decision_tree_vc}')

In [ ]:
### Decision Tree com GridSearchCV

In [ ]:
decision_tree_params = {'criterion': ['gini', 'entropy'],
                        'splitter': ['best', 'random'],
                        'max_depth': [None, 5, 10, 15, 20],
                        'min_samples_split': [2, 5, 10]}

modelo, acc_decision_tree, acc_decision_tree_cv, recall_decision_tree, f1_decision_tree = func_acuracia_GS(
    DecisionTreeClassifier(), decision_tree_params, X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia Decision Tree: {acc_decision_tree}')
print(f'Acurácia Decision Tree com Validação Cruzada: {acc_decision_tree_cv}')
print(f'Recall Decision Tree: {recall_decision_tree}')
print(f'F1-Score Decision Tree: {f1_decision_tree}')

In [ ]:
### Random Forest

In [ ]:
acc_random_forest, acc_random_forest_vc = func_acuracia(RandomForestClassifier(n_estimators=100), X, y, 10)

print(f'Acurácia Random Forest: {acc_random_forest}')
print(f'Acurácia Random Forest com Validação Cruzada: {acc_random_forest_vc}')

In [ ]:
### Random Forest com GridSearchCV

In [ ]:
random_forest_params = {'n_estimators': [50, 100, 200],
                        'criterion': ['gini', 'entropy'],
                        'max_depth': [None, 5, 10, 15, 20],
                        'min_samples_split': [2, 5, 10]}

modelo, acc_random_forest, acc_random_forest_cv, recall_random_forest, f1_random_forest = func_acuracia_GS(
    RandomForestClassifier(), random_forest_params, X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia Random Forest: {acc_random_forest}')
print(f'Acurácia Random Forest com Validação Cruzada: {acc_random_forest_cv}')
print(f'Recall Random Forest: {recall_random_forest}')
print(f'F1-Score Random Forest: {f1_random_forest}')

In [ ]:
### Gradient Boosting Classifier

In [ ]:
acc_gradient_boosting, acc_gradient_boosting_vc = func_acuracia(GradientBoostingClassifier(), X, y, 10)

print(f'Acurácia Gradient Boosting Classifier: {acc_gradient_boosting}')
print(f'Acurácia Gradient Boosting Classifier com Validação Cruzada: {acc_gradient_boosting_vc}')

In [ ]:
### Gradient Boosting Classifier com GridSearchCV

In [ ]:
gradient_boosting_params = {'loss': ['deviance', 'exponential'],
                            'learning_rate': [0.01, 0.1, 1],
                            'n_estimators': [50, 100, 200],
                            'criterion': ['friedman_mse', 'mse', 'mae'],
                            'max_depth': [None, 5, 10, 15, 20],
                            'min_samples_split': [2, 5, 10]}

modelo, acc_gradient_boosting, acc_gradient_boosting_cv, recall_gradient_boosting, f1_gradient_boosting = func_acuracia_GS(
    GradientBoostingClassifier(), gradient_boosting_params, X, y, 10)
print(f'Parâmetros otimizados do Modelo: {modelo}')
print(f'Acurácia Gradient Boosting Classifier: {acc_gradient_boosting}')
print(f'Acurácia Gradient Boosting Classifier com Validação Cruzada: {acc_gradient_boosting_cv}')
print(f'Recall Gradient Boosting Classifier: {recall_gradient_boosting}')
print(f'F1-Score Gradient Boosting Classifier: {f1_gradient_boosting}')

In [ ]:
### CatBoost

In [ ]:
acc_catboost, acc_catboost_vc = func_acuracia(CatBoostClassifier(verbose=False), X, y, 10)

print(f'Acurácia CatBoost: {acc_catboost}')
print(f'Acurácia CatBoost com Validação Cruzada: {acc_catboost_vc}')

In [ ]:
### CatBoost com GridSearchCV (cancelado pois demorou muito)

In [ ]:
##catboost_params = {'iterations': [50, 100, 200],
##                   'learning_rate': [0.01, 0.1, 1],
##                   'depth': [None, 5, 10, 15, 20]}
##
##modelo, acc_catboost, acc_catboost_cv, recall_catboost, f1_catboost = func_acuracia_GS(
##    CatBoostClassifier(verbose=True),
##    catboost_params, X, y, 3)
##print(f'Parâmetros otimizados do Modelo: {modelo}')
##print(f'Acurácia CatBoost: {acc_catboost}')
##print(f'Acurácia CatBoost com Validação Cruzada: {acc_catboost_cv}')
##print(f'Recall CatBoost: {recall_catboost}')
##print(f'F1-Score CatBoost: {f1_catboost}')

In [ ]:
### Comparando os modelos

In [ ]:
recall_catboost = 0
f1_catboost = 0

models = pd.DataFrame({
    'Modelo': ['Regressão Logística', 'KNN', 'Gaussian Naive Bayes', 'Linear SVC', 'Stochastic Gradient Descent',
               'Decision Tree', 'Random Forest', 'Gradient Boosting Classifier', 'CatBoost'],
    'Acurácia': [acc_log, acc_knn, acc_gaussian, acc_linear_svc, acc_sgd, acc_decision_tree, acc_random_forest,
                 acc_gradient_boosting, acc_catboost],
    'Acurácia com Validação Cruzada': [acc_log_vc, acc_knn_vc, acc_gaussian_vc, acc_linear_svc_vc, acc_sgd_vc,
                                       acc_decision_tree_vc, acc_random_forest_vc, acc_gradient_boosting_vc,
                                       acc_catboost_vc],
    'Recall': [recall_log, recall_knn, recall_gaussian, recall_svc, recall_sgd, recall_decision_tree,
               recall_random_forest, recall_gradient_boosting, recall_catboost],
    'F1-Score': [f1_log, f1_knn, f1_gaussian, f1_svc, f1_sgd, f1_decision_tree, f1_random_forest, f1_gradient_boosting,
                 f1_catboost]})

In [ ]:
models